<a href="https://colab.research.google.com/github/Belebez/VK_test/blob/main/vk_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Спасибо, что смотрите 2 вариант.

Здесь буду использовать коллаборативную систему (collaborative filtering). А именно основанную на предмете рекомендации (item-based).



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Этап 1. Подготовка данных



In [3]:
import pandas as pd
import numpy as np

# модуль sparse библиотеки scipy понадобится для работы с разреженными матрицами
from scipy.sparse import csr_matrix

# импортируем алгоритм k-ближайших соседей
from sklearn.neighbors import NearestNeighbors

Для начала прочитаем внешние файлы

In [4]:
# прочитаем файлы
movies = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/task_vk/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/task_vk/ratings.csv')

In [5]:
# посмотрим на содержимое файла movies.csv
# дополнительно удалим столбец genres, он не нужен
movies.drop(['genres'], axis = 1, inplace = True)

In [7]:
movies.head(10)

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
5,6,Heat (1995)
6,7,Sabrina (1995)
7,8,Tom and Huck (1995)
8,9,Sudden Death (1995)
9,10,GoldenEye (1995)


In [6]:
# и ratings.csv (здесь также удаляем ненужный столбец timestamp)
ratings.drop(['timestamp'], axis = 1, inplace = True)

In [8]:
ratings.head(3)

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5


In [9]:
data = pd.merge(movies, ratings)

In [10]:
data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),4,4.0
1,1,Toy Story (1995),10,5.0
2,1,Toy Story (1995),14,4.5
3,1,Toy Story (1995),15,4.0
4,1,Toy Story (1995),22,4.0


In [11]:
data.shape

(27753444, 4)

In [12]:
# исключим фильмы, которые набрали меньше 7000 голосов
# с этим количеством нормально работает colab

votes_counts = pd.DataFrame(data["title"].value_counts())
rare_movies = votes_counts[votes_counts["title"] <= 7000].index
selection_movies = data[~data["title"].isin(rare_movies)]

In [13]:
selection_movies.shape

(17031048, 4)

Создадим матрицу предпочтений 

In [14]:
# по горизонтали будут фильмы, по вертикали - пользователи, значения - оценки
# с помощью функции fillna() мы заменим NaN на ноль.
user_item_matrix = selection_movies.pivot(index = 'movieId', columns = 'userId', values = 'rating').fillna(0)
user_item_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,283219,283220,283221,283222,283223,283224,283225,283226,283227,283228
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
2,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


В датасете очень много нулей. Такая матрица называется разреженной. Одновременно, если столбцов очень много, то говорят про данные с высокой размерностью. В таком формате алгоритм будет долго обсчитывать расстояния между фильмами.

Для того, чтобы преодолеть эту сложность можно преобразовать данные в формат сжатого хранения строкой

In [30]:
# преобразуем разреженную матрицу в формат csr
# метод values передаст функции csr_matrix только значения датафрейма
csr_data = csr_matrix(user_item_matrix.values)

In [33]:
# посмотрим на записи
print(csr_data[:2,:5])

  (0, 0)	1.0
  (0, 4)	4.0
  (1, 0)	2.0
  (1, 4)	4.0


In [16]:
# сбросим индекс для удобства поиска фильма по индексу
user_item_matrix = user_item_matrix.rename_axis(None, axis = 1).reset_index()
user_item_matrix.head()

,movieId,1,2,3,4,5,6,7,8,9,...,283219,283220,283221,283222,283223,283224,283225,283226,283227,283228
0,1,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5
1,2,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Этап 2. Обучение модели

Рассчитаем расстояния от каждого фильма до ближайших векторов (других фильмов).

In [17]:
# воспользуемся классом NearestNeighbors для поиска расстояний
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute', n_neighbors = 20, n_jobs = -1)

# обучим модель
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

Параметры:

 - metric = ‘cosine’: выбираем способ измерения расстояния, в нашем случае это будет косинусное сходство
 - algorithm = ‘brute’: предполагает, что мы будем искать решение методом полного перебора (brute force search), в данном случае пространство решений позволяет перебрать все варианты
 - n_neighbors = 20: по скольким соседям ведется обучение
 - n_jobs = -1: в этом случае предполагается, что вычисления будут вестись на всех свободных ядрах процессора

### Этап 3. Составление рекомендаций

Зададим изначальные параметры поиска

In [18]:
# сколько рекомендаций мы хотим получить, пусть будет 7
recommendations = 7

# на основе, одного из моего любимого фильма
search_word = 'Interstellar'

Найдем индекс фильма в матрице предпочтений

In [19]:
# для начала найдем фильм в заголовках датафрейма movies
movie_search = movies[movies['title'].str.contains(search_word)]
movie_search

,movieId,title
23262,109487,Interstellar (2014)


In [20]:
# вариантов может быть несколько, для простоты всегда будем брать первый вариант
# через iloc[0] мы берем первую строку столбца ['movieId']
movie_id = movie_search.iloc[0]['movieId']

# далее по индексу фильма в датасете movies найдем соответствующий индекс
# в матрице предпочтений
movie_id = user_item_matrix[user_item_matrix['movieId'] == movie_id].index[0]
movie_id

980

Находим схожие фильмы

In [21]:
# теперь нужно найти индексы и расстояния фильмов, которые похожи на наш запрос
# воспользуемся методом kneighbors()
distances, indices = knn.kneighbors(csr_data[movie_id], n_neighbors = recommendations + 1)

In [22]:
# индексы рекомендованных фильмов
indices

array([[980, 941, 996, 990, 989, 983, 971, 959]])

In [23]:
# расстояния до них
distances

array([[0.        , 0.41188006, 0.42648806, 0.44141406, 0.45760804,
        0.45762732, 0.45794665, 0.46532205]])

In [25]:
# уберем лишние измерения через squeeze() и преобразуем массивы в списки с помощью tolist()
indices_list = indices.squeeze().tolist()
distances_list = distances.squeeze().tolist()

# далее с помощью функций zip и list преобразуем наши списки
indices_distances = list(zip(indices_list, distances_list))

# и посмотрим на первые три пары/кортежа
indices_distances[:3]

[(980, 0.0), (941, 0.4118800600367596), (996, 0.4264880606040634)]

In [26]:
# остается отсортировать список по расстояниям через key = lambda x: x[1] (то есть по второму элементу)
# в возрастающем порядке reverse = False
indices_distances_sorted = sorted(indices_distances, key = lambda x: x[1], reverse = False)

# и убрать первый элемент, так как это выбранный фильм
indices_distances_sorted = indices_distances_sorted[1:]
indices_distances_sorted

[(941, 0.4118800600367596),
 (996, 0.4264880606040634),
 (990, 0.4414140606504777),
 (989, 0.45760804416835466),
 (983, 0.4576273231345368),
 (971, 0.45794665208638574),
 (959, 0.4653220493844601)]

Остается найти какие фильмы соответствуют найденным нами индексам

In [27]:
# создаем пустой список, в который будем помещать название фильма и расстояние до него
recom_list = []

# теперь в цикле будем поочередно проходить по кортежам
for ind_dist in indices_distances_sorted:

    # искать movieId в матрице предпочтений
    matrix_movie_id = user_item_matrix.iloc[ind_dist[0]]['movieId']

    # выяснять индекс этого фильма в датафрейме movies
    id = movies[movies['movieId'] == matrix_movie_id].index

    # брать название фильма и расстояние до него
    title = movies.iloc[id]['title'].values[0]
    dist = ind_dist[1]

    # помещать каждую пару в питоновский словарь
    # который, в свою очередь, станет элементом списка recom_list
    recom_list.append({'Title' : title, 'Distance' : dist})

In [28]:
# посмотрим на первый элемент
recom_list[0]

{'Title': 'Inception (2010)', 'Distance': 0.4118800600367596}

In [29]:
# остается преобразовать наш список в датафрейм
# индекс будем начинать с 1
recom_df = pd.DataFrame(recom_list, index = range(1, recommendations + 1))
recom_df

,Title,Distance
1,Inception (2010),0.411880
2,The Martian (2015),0.426488
3,The Imitation Game (2014),0.441414
4,Ex Machina (2015),0.457608
5,Edge of Tomorrow (2014),0.457627
6,Django Unchained (2012),0.457947
7,"Dark Knight Rises, The (2012)",0.465322


Еще раз хорошего настороения)

